[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YakivGalkin/cnn_decision_transformer/blob/main/record_video.ipynb)


In [19]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
  if not os.path.exists('/content/.already_installed'):
    !git clone https://github.com/YakivGalkin/cnn_decision_transformer
    !apt-get install -y swig
    !pip install -r cnn_decision_transformer/requirements.txt
    with open('/content/.already_installed', 'w') as f:
        f.write('done')
  %cd /content/cnn_decision_transformer

In [20]:
import os
from dataclasses import dataclass
import gymnasium as gym
import numpy as np
import torch
from stable_baselines3 import DQN
from stable_baselines3.common.torch_layers import NatureCNN
from stable_baselines3.common.vec_env import DummyVecEnv, VecTransposeImage

In [21]:
import utils.storage as storage
loaded_model = storage.get_pretrained_model(DQN,  'dql_car_racing.pt')

/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: Can't get attribute '_make_function' on <module 'cloudpickle.cloudpickle' from '/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/cloudpickle/cloudpickle.py'>
  warnings.warn(
/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: Can't get attribute '_make_function' on <module 'cloudpickle.cloudpickle' from '/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/cloudpickle/cloudpickle.py'>
  warnings.warn(
/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:231: UserWarning: This sy

In [22]:
from visual_decision_transformer.cnn_feature_extractor import prepare_env_observation_array


env =  gym.make('CarRacing-v2', render_mode="rgb_array", continuous=False)

MAX_EPISODE_STEPS = 1000
CUT_IF_NO_REWARD = 50


for episode in range(1):
    print(f"Episode: {episode+1} of {1}:" )
    [obs, _] = env.reset()
    done = False

    o, r = [], []
    render_arr = []
    total_reward = 0
    step_index = 0
    while not done:
        step_index +=  1

        action, _states = loaded_model.predict(obs,deterministic=True)
        new_obs, reward, done, t, i = env.step(action)
        total_reward = total_reward + reward

        o.append(obs)

        r.append(reward)
        render_arr.append( env.render())
        obs = new_obs
        print(".", end="")

        # any positive reward within last 50 steps?
        if step_index > MAX_EPISODE_STEPS or (len(r) > CUT_IF_NO_REWARD and max(r[-CUT_IF_NO_REWARD:]) <= 0):
            break

    fin_positive = len(r)-1
    for fin_positive in range (len(r)-1, 0, -1):
        if r[fin_positive] > 0:
            break

    r = r[:fin_positive+1]
    o = o[:fin_positive+1]

    print(f"\nTotal reward: {total_reward} steps: {len(o)}")
env.close()



Episode: 1 of 1:
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [23]:
%mkdir recorded_videos

mkdir: recorded_videos: File exists


In [24]:
import imageio
import numpy as np

# Let's assume `frames` is your list of NumPy arrays
# frames = [numpy_array1, numpy_array2, ..., numpy_arrayN]

# Define the path for the output video file
output_path = 'recorded_videos/output_video_rm7.mp4'

# Create the writer object
#writer = imageio.get_writer(output_path, fps=30)

# Iterate over each frame and add it to the video
with imageio.get_writer(output_path, fps=30, quality=5, codec='libx264') as writer:
    for frame in render_arr:
        # Check if the frame is uint8, if not, convert it
        if frame.dtype != np.uint8:
            frame = frame.astype(np.uint8)
        writer.append_data(frame)

# Don't forget to close the writer to finish writing the video file
writer.close()

print(f"Video saved to {output_path}")


Video saved to recorded_videos/output_video_rm7.mp4


In [25]:
from IPython.display import HTML
from base64 import b64encode

# Path to your MP4 video
video_path = output_path

# Encode the video to base64 (if the video is large, this operation may take a while)
video_encoded = b64encode(open(video_path, "rb").read()).decode()

# HTML5 video tag for embedding
video_tag = f'<video width="640" height="480" controls><source src="data:video/mp4;base64,{video_encoded}" type="video/mp4"></video>'

# Display the video in the notebook
HTML(video_tag)